In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = (8, 6) # set default figure size, 8in by 6in

# Video W2 01: Multiple Features

[YouTube Video Link](https://www.youtube.com/watch?v=Zn0u3mKrBEo&list=PLZ9qNFMHZ-A4rycgrgOYma6zxF4BZGGPW&index=19)

In the previous week, we only looked at a toy example of linear regression with a single feature, which
ended up needing two parameters to be fitted to create a hypothesis or model of the data.

The technique that we looked at, and the equations we developed, for linear regression are completely
generalizable to as many input features as are available that we would like to fit.

Notationally, we had been simply using $x$ to denote the input, and $y$ to denote the output.  Now
that we have more features, we will need a bit of additional notation.  Recall that we used $x^{(i)}$
to denote the $i^{th}$ input and output training example.  However, now instead of having a single
feature (like size in square feet), we have $n$ number of features.  We will use the expanded notation:

**Notation:**

- $n = $ number of features
- $x^{(i)} = $ input (features) of the $i^{th}$ training example (this is a vector of length $n$ ).
- $x_j^{(i)} = $ value of particular feature $j$ in the $i^{th}$ training example.
- $m = $ number of training examples.
- $y^{(i)} = $ output of the $i^{th}$ training example.

Using `NumPy` array indexing, the first index will select the row, which corresponds to the input
pattern number $i$, and the second index will select the column, which corresponds to the feature
number $j$.  Here is an example:

In [2]:
house = pd.read_csv('data/housing-prices-4-features-portland-or.csv')
x = house.iloc[:,0:4].as_matrix()
y = house.price.as_matrix()

print(x.shape)
print(y.shape)

(47, 4)
(47,)


In [3]:
# the first 4 training input patterns
print(x[:4])

[[2104    3    1   45]
 [1600    3    2   40]
 [2400    3    2   30]
 [1416    2    1   36]]


In [4]:
# the input training pattern at index 3 (0 based indexing, so actually the 4th overall
print(x[3])

[1416    2    1   36]


In [5]:
# feature 0 is the size, feature 1 is the number of bedrooms, feature 2 is number of floors,
# feature 3 is the age of the house
print("size (sq ft.) = ", x[3,0])
print("num bedrooms  = ", x[3,1])
print("num floors    = ", x[3,2])
print("age of house  = ", x[3,3])

size (sq ft.) =  1416
num bedrooms  =  2
num floors    =  1
age of house  =  36


What is the form of the hypothesis with more than 1 feature. For 4 features we would have:

$$h_\theta(x) = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + \theta_3 x_3 + \theta_4 x_4$$

And in general, for $n$ features, we have $n + 1$ parameters:

$$h_\theta(x) = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + ... + \theta_n x_n$$

For convenience in programming, we usually think of the previous equation as having
an additional input $x_0 = 1$ that is always set to one.  In this way, we can define
$n + 1$ sized vectors to represent some particular set of input features, and another 
$n + 1$ sized vector to represent a set of $\theta$ hypothesis parameters:

$$
x =
\begin{bmatrix}
x_0 \\
x_1 \\
x_2 \\
\vdots \\
x_n \\
\end{bmatrix}
\in \mathbb{R}^{n+1}
\;\;\;
\theta =
\begin{bmatrix}
\theta_0 \\
\theta_1 \\
\theta_2 \\
\vdots \\
\theta_n \\
\end{bmatrix}
\in \mathbb{R}^{n+1}
$$

And a sort of a note: This is convenient for us using a 0 based indexing programming language like 
`NumPy` arrays, as we can use the $0^{th}$ index for the $x_0 = 1$ and the $\theta_0$ values.

Finally, the full form of our hypothesis for linear regression is:

$$h_\theta(x) = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... + \theta_n x_n$$

Using the transpose of the $\theta$ vector, we can use simple vector vector multiplication,
simplifying the hypothesis equation to be:

$$h_\theta(x) = \theta^T x$$

As an example, lets show how, given a set of 4 feature inputs, and a set of 4 hypothesis parameters,
we can get the hypothesis output using this simple vector vector multiplication:

In [6]:
x = np.array([[1.0], 
              [1416], 
              [3],
              [2], 
              [40]])
theta = np.array([[80],
                  [0.1],
                  [0.01],
                  [3],
                  [-2]])

print(np.dot(np.transpose(theta), x))

[[ 147.63]]


This result basically represents the prices (in 1000s of \$) for the given set of inputs, and given
the current set of hypothesis parameters.  Notice that we have 4 input features, and the $x_0$ feature
is the dummy value $1.0$.

# Video W2 02: Gradient Descent for Multiple Variables

[YouTube Video Link](https://www.youtube.com/watch?v=mmclAQ-UlbE&list=PLZ9qNFMHZ-A4rycgrgOYma6zxF4BZGGPW&index=20)

To summarize, here are our equations and notation for multivariate linear regression that
we are currently working with:

- **Hypothesis:** $h_\theta(x) = \theta^T x = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... + \theta_n x_n$
- **Parameters:** $\theta_0, \theta_1, \dots, \theta_n$
- **Cost function:**
$$J(\theta_0, \theta_1, \dots, \theta_n) = \frac{1}{2m} \sum_{i=1}^{m} \big( h_\theta(x^{(i)}) - y^{(i)} \big)^2$$
- **Gradient descent:**
`repeat` {
$$\theta_j := \theta_j - \alpha  \frac{\partial}{\partial \theta_j} J(\theta_0,..., \theta_n)$$
} (simultaneously update for every $j = 0, ..., n$

The cost function allows you to calculate the "cost", or how well a particular set of hypothesis
parameters $\theta$ does in modeling the data.  The lower the cost, the better the parameters are at
fitting the data.  The cost function sums differences squared between the hypothes model output
and the actual output.

The gradient descent algorithm is an algorithm we can use to find the values of the $\theta$ parameters
that give the best, or minimum cost, when fitting to the data.  When we were looking at 1 variable
last week, we only had two theta parameters.  For the multivariate case, we have to simultaneously
update the $n + 1$ $\theta$ parameters, but this extension to the basic idea is straight forward.

## Gradient Descent update Equations

The generalized case for mutivariate gradient descent looks like the following.  This equation
basically has determined the partial derivative of the $J()$ cost function with respect to each
variable:


`repeat` {
$$\theta_j := \theta_j - \alpha  \frac{1}{m} \sum_{i=1}^m \big(h_\theta(x^{(i)}) - y^{(i)} \big) x_j^{(i)} $$
} (simultaneously update for every $j = 0, ..., n$


Two things to note about this form for our gradient descent algorithm:

1. $x_0$ will always be 1 by definition, so this form reduces to the form we gave in the univariate 
   case.
2. The partial derivative becomes the result of a summation of the difference between the
   hypothesis output and the actual output (divided by $m$, so you get an average over all
   $m$ input patterns), multiplied by the particular value of the feature for each pattern.  The
   only thing that differs in each case is the particular feature, which is a result of taking
   the partial derivative with respect to that feature.
   

# Video W2 03: Gradient Descent in Practice I: Feature Scaling

[YouTube Video Link](https://www.youtube.com/watch?v=yQci-wS0iMw&list=PLZ9qNFMHZ-A4rycgrgOYma6zxF4BZGGPW&index=21)

Scaling features can help make sure gradient descent converges at a much faster pace.  Generally
scaling means in practice we will try and get all features to be in the range $-1 \le x_i \le 1$.

Scaling can be done in 2 steps:

1. Center the data (mean normalization), so features have approximately have 0 mean.
2. Scale the data to a common range (scaling).

Here is a quick example for our house data:

In [7]:
house = pd.read_csv('data/housing-prices-4-features-portland-or.csv')
house.insert(0, 'default', '1.0') # insert column at index 0, the 1.0 default values
x = house.iloc[:,0:5].as_matrix()
y = house.price.as_matrix()

In [8]:
# mean normalization, center the data around 0

# the house size data is in x[:,1]
print(x[:,1])

# the mean value of the house data is
print("Mean before centering:", np.mean(x[:,1]))
print("")

# do the actual centering
x[:,1] = x[:,1] - np.mean(x[:,1])

print(x[:,1])
print("Mean after centering:", np.mean(x[:,1]))

[2104 1600 2400 1416 3000 1985 1534 1427 1380 1494 1940 2000 1890 4478 1268
 2300 1320 1236 2609 3031 1767 1888 1604 1962 3890 1100 1458 2526 2200 2637
 1839 1000 2040 3137 1811 1437 1239 2132 4215 2162 1664 2238 2567 1200 852
 1852 1203]
Mean before centering: 2000.6808510638298

[103.31914893617022 -400.6808510638298 399.3191489361702 -584.6808510638298
 999.3191489361702 -15.680851063829778 -466.6808510638298
 -573.6808510638298 -620.6808510638298 -506.6808510638298
 -60.68085106382978 -0.6808510638297776 -110.68085106382978
 2477.31914893617 -732.6808510638298 299.3191489361702 -680.6808510638298
 -764.6808510638298 608.3191489361702 1030.3191489361702
 -233.68085106382978 -112.68085106382978 -396.6808510638298
 -38.68085106382978 1889.3191489361702 -900.6808510638298
 -542.6808510638298 525.3191489361702 199.31914893617022 636.3191489361702
 -161.68085106382978 -1000.6808510638298 39.31914893617022
 1136.3191489361702 -189.68085106382978 -563.6808510638298
 -761.6808510638298 131.

In [9]:
# scaling, the data ranges from the minimum value up to the maximum value, if we divide by
# this range, we will scale the data to values between -1.0 and 1.0
minimum = min(x[:,1])
maximum = max(x[:,1])
rnge = maximum - minimum
print(rnge)

x[:,1] = x[:,1] / rnge
print(x[:,1])

3626.0
[0.028493973782727586 -0.11050216521341141 0.11012662684395208
 -0.161246787386605 0.27559822088697467 -0.004324559035805234
 -0.1287040405581439 -0.15821314149581628 -0.17117508302918638
 -0.13973548016101206 -0.01673492858903193 -0.0001877691847296684
 -0.030524228092617147 0.6832099142129537 -0.20206311391721726
 0.08254802783678164 -0.18772224243348865 -0.2108882655995118
 0.1677658987689383 0.28414758657919753 -0.06444590487143678
 -0.031075800072760555 -0.10939902125312459 -0.010667636807454434
 0.5210477520507916 -0.24839516024926359 -0.14966377580359344
 0.14487566159298682 0.0549694288296112 0.175487906490946
 -0.04458931358627407 -0.275973759256434 0.010843670418138506
 0.3133809015267982 -0.05231132130828179 -0.15545528159509922
 -0.2100609076292967 0.0362159815047353 0.6106781988240955
 0.044489561206886435 -0.09285186184882233 0.06544929645233596
 0.1561828871859267 -0.22081656124209315 -0.31679008578704626
 -0.041004095715341915 -0.21998920327187804]


# Video W2 04: Gradient Descent in Practice II: Learning Rate

[YouTube Video Link](https://www.youtube.com/watch?v=mlHcA-VCyN0&list=PLZ9qNFMHZ-A4rycgrgOYma6zxF4BZGGPW&index=22)

The take away from this video discussion:

- If $\alpha$ is too small: slow convergence
- If $\alpha$ is too large: $J(\theta)$ may not decrease on every iteration; may not converge.

In practice, try running with a range of $\alpha$ learning rate values and plot the cost as a function
of the number of iterations.  You want to find values for the learning rate where the cost converges
and is reduced at a good rate.

# Video W2 05: Features and Polynomial Regression

[YouTube Video Link](https://www.youtube.com/watch?v=8p3S9aR3n4o&index=23&list=PLZ9qNFMHZ-A4rycgrgOYma6zxF4BZGGPW)

Some discussions here on how you can make linear regression more powerful.  The main idea is that
you can actually combine or choose features.  This does not effect the cost function nor calculating
gradient descent, but it can sometimes be used to find models for data that have complex, nonlinear
features.

# Video W2 06: Normal Equation

[YouTube Video Link](https://www.youtube.com/watch?v=rnlti7rgns0&list=PLZ9qNFMHZ-A4rycgrgOYma6zxF4BZGGPW&index=24)

The normal equation is basically an analytical solution to solve for the optimal $\theta$ parameters.
It has some advantages and disadvantages over using the gradient descent algorithm.  Here is the
normal equation:

$$
\theta = (X^T X)^{-1} X^T y
$$



In [10]:
X = np.array([[1.0, 2104, 5, 1, 45],
              [1.0, 1416, 3, 2, 40],
              [1.0, 1534, 3, 2, 30],
              [1.0,  852, 2, 1, 36]])

y = np.array([[460],
              [232],
              [315],
              [178]])

# these lines implement (X^T X)^-1 X^T y
# this is the Normal Equation, the values given for Theta are the minimum
# fitted parameters for the given data
Tmp = np.linalg.inv(np.dot(X.T, X))
Theta = np.dot(np.dot(Tmp, X.T), y)
print(Theta)

[[  4.36875000e+02]
 [  2.87841797e-01]
 [ -9.62500000e+01]
 [ -9.00625000e+01]
 [ -4.50390625e+00]]


This method is basically the same as solving a series of simultaneous equations, where you set each
equation to 0.  

What are the advantages of and disadvantates of gradient descent vs. solving analytically using the
normal equation?

Gradient descent requires you to figure out a good $\alpha$ learning rate, and can require many iterations
to converge (so might take some time).  The normal equation does not have any parameter like the learning
rate that you have to figure out, and it does not require any iterations.

However, the need to compute the inverse of a matrix is itself a very time costly algorithm for a computer, so for very large numbers of features, inverting the matrix might end up taking much longer
than gradient descent.  So gradient descent works well, even if you have very large numbers of features.
Thus for Big Data problems, that can have thousands or millions of features, you almost always use the
iterative gradient descent.

# Video W2 07: Normal Equation and Non-Invertibility (Optional)

[YouTube Video Link](https://www.youtube.com/watch?v=t-5Take484A&list=PLZ9qNFMHZ-A4rycgrgOYma6zxF4BZGGPW&index=25)

Only some matrices are invertible.  Those that are not invertible are called singular or degenerate
matrices.  Intuitively, you can think of these matrices as being in some sense close to the value 0. 
The scalar value is not invertible, and such degenerate matrices are not invertible for a similar reason.
Common causes for a matrix to be non-invertible are:

1. Redundant features (that are linearly dependent)
2. Too many features (e.g. $m \le n$)


In [11]:
%load_ext version_information

%version_information numpy, scipy, matplotlib, pandas, sklearn

Software versions
Python 3.6.1 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython 6.4.0
OS Linux 4.13.0 46 generic x86_64 with debian stretch sid
numpy 1.12.1
scipy 0.19.0
matplotlib 2.0.2
pandas 0.20.1
sklearn 0.18.1
Sun Aug 26 11:07:30 2018 CDT